In [ ]:
import pandas as pd
import sqlite3


In [ ]:
print("\n[ S1 ] Reading Orders File\n")

orders_data = pd.read_csv("orders.csv")
print(orders_data.head())

print("\nOrders Columns:")
print(list(orders_data.columns))

print("\nOrders Dataset Summary:")
orders_data.info()


In [ ]:
print("\n[ S2 ] Reading Users File\n")

users_data = pd.read_json("users.json")
print(users_data.head())

print("\nUsers Columns:")
print(list(users_data.columns))

print("\nUsers Dataset Summary:")
users_data.info()


In [ ]:
print("\n[ S3 ] Loading Restaurant Data\n")

db_conn = sqlite3.connect("restaurants.db")
db_cursor = db_conn.cursor()

# remove old table if exists
db_cursor.execute("DROP TABLE IF EXISTS restaurants")

with open("restaurants.sql", "r") as sql_file:
    sql_script = sql_file.read()
    db_cursor.executescript(sql_script)

db_conn.commit()

restaurant_data = pd.read_sql(
    "SELECT * FROM restaurants",
    db_conn
)

print(restaurant_data.head())
print("\nRestaurant Dataset Summary:")
restaurant_data.info()

In [ ]:
print("\n[ S4 ] Combining All Tables\n")

orders_users_merge = pd.merge(
    orders_data,
    users_data,
    on="user_id",
    how="left"
)

complete_dataset = pd.merge(
    orders_users_merge,
    restaurant_data,
    on="restaurant_id",
    how="left"
)

print(complete_dataset.head())
print("\nMerged Dataset Overview:")
complete_dataset.info()

In [ ]:
print("\n[ S5 ] Cleaning Columns & Exporting\n")

if "restaurant_name_x" in complete_dataset.columns:
    complete_dataset.drop("restaurant_name_x", axis=1, inplace=True)

complete_dataset.rename(
    columns={"restaurant_name_y": "restaurant_name"},
    inplace=True
)

complete_dataset["order_date"] = pd.to_datetime(
    complete_dataset["order_date"],
    dayfirst=True
)

complete_dataset.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

print("✅ Dataset created: final_food_delivery_dataset.csv")